# Final Code

<div class="alert alert-success">
All of the concepts on choosing the subsets and data particularities in this code is concluded from the 'EDA - analysing Nans' notebook, which is also included in this repository. <br> <br>

</div>


# Dataset tuning

All of this steps below are based on the EDA notebook also present in this repository.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Math, Latex
from scipy import stats, linalg
% matplotlib inline


df = pd.read_csv('SPcities2.csv', decimal=',', encoding='Iso-8859-1', sep=';')

## Seting the data as seeing in the EDA notebook ( also in this repository)
df = df[df.year > 2001]
df = df[df.year < 2016]

data = df[df.year == 2013] # like we saw in the EDA on missing values, 2013 should be the best year for our analysis
                           # this was the case because, for this method, we need the year with more information, 
                            # in this case 2013

# cleaning the all null features

data_clean = data
data_clean = data_clean.drop('HDI', 1)
data_clean = data_clean.drop('educ_superior', 1)
data_clean = data_clean.drop('tax_revenue', 1)

------------------------------------------------ END OF DATASET TUNING ----------------------------------------------

The I-Distance method:

<img src='idistance.png'> </img>

<img src='disceffect.png'> </img>

"where di(r, s) is the distance between the values of variable Xi for er and es e.g. the discriminate effect" and <br> <br>

"σi the standard deviation of Xi, and rji.12 … j − 1 is a partial coefficient of the correlation between Xi and Xj, (j < i)"

# Creating the tools to use the method

In [9]:
# this idea for partial correlation was obtained from : https://gist.github.com/fabianp/9396204419c7b638d38f

## Partial Correlation Matrix for the rij variable in the formula

from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr

        
    return P_corr

In [10]:
# The reference hyperplane to calculate all the distances from.
# Here we are going to set the city with the smallest value for GDP per capita

data_clean.sort_values('gdp_per', ascending=True).head()

,city,year,gdp_per,gdp_growth,private_inv,pub_inv,export,import,violence,primary_enrolls,...,hosp_rooms,hosp_rooms_per,jobs,jobs_revenue,population,urban_pop,rural_pop,olding,urbing,fundamental
4450,Itapirapuã Paulista,2013,6956.90,0.098176,NaN,917044.27,NaN,NaN,12,102.25,...,NaN,NaN,521.0,1266.38,3954,1959,1995.0,42.32,49.54,64.0
10196,Taquaral,2013,7307.11,-0.110965,NaN,91200.00,NaN,NaN,9,89.24,...,NaN,NaN,500.0,1159.92,2729,2623,106.0,74.68,96.12,34.0
8462,Riversul,2013,7386.98,0.132233,NaN,NaN,NaN,NaN,72,94.24,...,NaN,NaN,533.0,1188.74,6003,4423,1580.0,76.79,73.68,64.0
7816,Pracinha,2013,7650.01,0.210545,NaN,NaN,NaN,NaN,7,99.51,...,NaN,NaN,264.0,1307.56,2864,1372,1492.0,77.06,47.91,17.0
285,Álvaro de Carvalho,2013,7653.97,-0.031318,NaN,1162680.60,NaN,NaN,29,100.62,...,NaN,NaN,359.0,1443.34,4763,3081,1682.0,60.52,64.69,55.0


Here, since the city of Itapirapua Paulista has the smallest value for the features of our most interest, it will be the hyperplane reference for our calculations.

In [11]:
data_values = data_clean.drop('year' ,1).fillna(0) # Base dataset for all calculations


reference = data_values.sort_values('gdp_per', ascending=True).iloc[0:1,:] # Base reference for all calculations

# Building the function to calculate the I-distance

Like the steps followed by <a href="http://www.sciencedirect.com/science/article/pii/S0264999314000558">reference article </a> , we should : <br>

<ol>
<li>Calculate the value of the discriminate effect of the variable X1, the most significant variable, that which provides the largest amount of information on the phenomena that is to be ranked; </li>
<li> Add the value of the discriminate effect of X2 which is not covered by X1;</li>
<li> Add the value of the discriminate effect of X3 which is not covered by X1 and X2;</li>
<li> Repeat the procedure for all variables;</li>
<li> Calculte the correlation coefficient of each indicator with the I-distance;</li>
<li> Exclude the smallest non significant coefficient;</li>
<li> Repeat (6) until only significant features remain.</li>


In [27]:
## Creating a function to the calculation to operate 1-4 above: 

def I_distance(data_values, reference):
    
    #getting the correlation matrix done
    data_corr = data_values.drop('city' ,1).fillna(0)
    col_labels = list(data_corr.columns)
    corre = pd.DataFrame(partial_corr(data_corr), columns=col_labels)
                         
    #starting the I-Distance calculation                     
    ncolums = data_corr.shape[1]
    nrows = data_corr.shape[0]
    somatory = []
    i2_index = {'city': [], 'I-Distance': []}
    for r in range(0,nrows):
        somatory = []    
        for i in range(0,ncolums):
            prod = 1
            summ_1 = 0
            summ_1 = ((data_corr.iloc[r,i] - reference.iloc[0,i+1])**2)/np.var(data_corr.iloc[:,i]) #distance squared divided by variance
            for k in range(i+1):
                if corre.iloc[i,k] == 1:
                    prod *= 1
                else:
                    prod *= (1-(corre.iloc[i,k])**2)
            somatory.append(summ_1*prod)
        i2_index['city'].append(data_values.iloc[r,0])
        i2_index['I-Distance'].append(np.cumsum(somatory)[-1])
    

## 5 - 6 selecting sigficant features

# calculating the rank of features and p-values

    distance = pd.DataFrame(i2_index)

    data = data_values.iloc[:,1:]
    feature_rank = {'feature': [], 'r': [], 'p-value': []}

    # creating the rank with correlation, p-values and name of features
    for col, values in data.iteritems():
        feature_rank['feature'].append(col)
        feature_rank['r'].append(stats.pearsonr(values, distance['I-Distance'])[0])
        feature_rank['p-value'].append(stats.pearsonr(values, distance['I-Distance'])[1])
    

    # manipulating the rank to select smallest correlation value with no significant p-value valor (>0.01)
    rank = pd.DataFrame(feature_rank)
    lenght_rank = len(list(rank.columns))
  
    excluded = {'feature' : "" , 'p-value' : 0 }
    for x in range(lenght_rank):
        excluded = {'feature' : "" , 'p-value' : 0 }
        chk = 0
        if rank.sort_values('r').iloc[x,1] >= 0.05 and x <= lenght_rank:  # check if the smallest r value is significant
            excluded['feature'] = rank.sort_values('r').iloc[x,0]
            excluded['p-value'] = rank.sort_values('r').iloc[x,1]
            chk = 1
            break
        elif rank.sort_values('r').iloc[x,1] <= 0.05 and x == lenght_rank:
            chk = 0
            break
        else:
            chk = 0
            continue

    if chk == 1: # returning new data frame if there is some variable to exclude, otherwise is done
        n_values = data_values.drop(excluded['feature'], 1)
        n_ref = reference.drop(excluded['feature'], 1)
        return 'loop',distance,  n_values, n_ref, excluded
    else:
        return 'done', distance, rank, 'ref', excluded # if no features to drop, return distance and rank
        


In [49]:
# 7 - Iterating for the final model until we gett only significant features:

def Final_distance(data_values, reference):
    check, distance, rank, ref, excluded = I_distance(data_values, reference)
    
    cont=1
    while True:
        if check == 'loop':
            print("Iteration of Number: ", cont)
            print("Variable excluded in this iteration:", excluded )
            print("I-Distance rank:")
            print(distance)
            check, distance, rank, ref, excluded = I_distance(rank, ref)
            cont += 1
            continue
        else:
            print("Iterations needed: ", cont-1)
            break
    return check, distance, rank

In [48]:
check1, distance1, rank1 = Final_distance(data_values, reference)

Iteration of Number:  1
Variable excluded in this iteration: {'feature': 'olding', 'p-value': 0.62152333001401172}
I-Distance rank:
     I-Distance                    city
0     19.574375              Adamantina
1     12.623635                  Adolfo
2      7.619452                   Aguaí
3     14.912675          Águas da Prata
4     10.553639        Águas de Lindóia
5      4.973449  Águas de Santa Bárbara
6    175.278901      Águas de São Pedro
7     14.560906                  Agudos
8      2.860073                Alambari
9     13.647056       Alfredo Marcondes
10     7.707353                  Altair
11     9.946214             Altinópolis
12    16.617030             Alto Alegre
13    55.358300                Alumínio
14    20.329446        Álvares Florence
15     6.563823         Álvares Machado
16     1.786611      Álvaro de Carvalho
17     5.934405             Alvinlândia
18    24.343443               Americana
19    12.963261     Américo Brasiliense
20     8.488222       Améric

As we can see, of the features of our interest, public investment seems to be a important one, once it 'survived'and has as relevant participation on the index calculation. Otherwise, private investment seems to be irrelevant for economic growth in this particular case. <br><br>

Now, let's make some separate experiments and, using the same year of 2013, try to isolate our 2 variables of interest, public and private investment, and see how they perform without each others presence only in the city where they have not null entries. <b>This seeks to answer wheter, in the cities where they are important, what are their relevance in the I-Distance model. 

<h4>Public Investment: </h4>

In [ ]:
# let's prepare our dataset 

public = data_clean[data_clean.pub_inv.notnull()].drop(['year', 'private_inv'], 1).fillna(0) #this will be our dataset for this part

public.head(5)

In [ ]:
# Let`s get our reference, being the minimum value for GDP per capita, as we did above

pub_ref = public.sort_values('gdp_per', ascending=True).iloc[0:1,:]
pub_ref # it will be the same city, but we now have our reference

In [ ]:
checkpub1, distancepub1, rankpub1 = Final_distance(public, pub_ref)

In [30]:
rankpub1.sort_values('r', ascending=False)

NameError: name 'rankpub1' is not defined

It looks like the situation from our first run remains the same, so public investment is still relevant and in the same level, in the cities where it appears in the year of 2013. <br><br>

Let's take a look on the private investment:

<b> Private Investment

In [ ]:
# let's prepare our dataset 

private = data_clean[data_clean.private_inv.notnull()].drop(['year', 'pub_inv'], 1).fillna(0) #this will be our dataset for this part

private.head(5)

In [ ]:
# Let`s get our reference, being the minimum value for GDP per capita, as we did above

priv_ref = private.sort_values('gdp_per', ascending=True).iloc[0:1,:]
priv_ref # this is a different city from the other 2 situations

In [ ]:
checkpri1, distancepri1, rankpri1 = Final_distance(private, priv_ref)

In [ ]:
rankpri1.sort_values('r', ascending=False)

<div class="alert alert-success">
Here we could see that 2013 should be the better year to apply the method since we have the most of our main predictors. <br> <br>

Another thing is that, beginning from 2010 and going to 2013, could be a good period if we choose to use the mean of more than 1 years to calcule the hyperplanes, since we have important data concetrated in this years.  <br>
</div>